In [ ]:
################################################################################
!nvidia-smi

Sat Apr  3 19:22:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
################################################################################
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
################################################################################
!pip install  tensorflow-gpu
!pip install numpy
!pip install scikit-learn
!pip install scipy
!pip install simpleitk
!pip install grpcio
!pip install git+https://github.com/aleju/imgaug.git

In [ ]:
################################################################################
%matplotlib inline
import nibabel as nib
import tensorflow as tf
import datetime
import tensorflow as tf
print(tf.__version__)
import numpy as np
import h5py
import os
import random
from sklearn.utils import shuffle
from tqdm import tqdm
import math
import SimpleITK as sitk  # For loading the dataset
import numpy as np  # For data manipulation
import glob  # For populating the list of files
import re  # For parsing the filenames (to know their modality)
from skimage.transform import resize # For resizing
from scipy.ndimage import zoom  # For resizing but not used for now
import matplotlib.pyplot as plt
from PIL import Image
import imgaug.augmenters as iaa
from sklearn.model_selection import KFold


2.4.1


In [ ]:
################################################################################
def read_img(img_path):
    """
    Reads a .nii.gz image and returns as a numpy array.
    """
    
    #nib.load(img_path).get_data(caching='unchanged')
    return sitk.GetArrayFromImage(sitk.ReadImage(img_path))
  
def itensity_normalize_one_volume(volume):
    """
    normalize the itensity of an nd volume based on the mean and std of nonzeor region.
    inputs:
        volume: the input nd volume.
    outputs:
        out: the normalized nd volume.
    """
    if (volume > 0).any()!=0:
        pixels = volume[volume > 0]
        mean = pixels.mean()
        std  = pixels.std()
        if std!=0:
           out  = (volume - mean)/std
        else:
           out  = (volume - mean)/(std+0.0000001) 
    else:
        out  = volume
    out_random = np.zeros(volume.shape)
    out[volume == 0] = out_random[volume == 0]
    return out
  
def get_none_zero_region(im, margin):
    """
    get the bounding box of the non-zero region of an ND volume.
    """
    input_shape = im.shape
    if(type(margin) is int ):
        margin = [margin]*len(input_shape)
    assert(len(input_shape) == len(margin))
    indxes = np.nonzero(im)
    idx_min = []
    idx_max = []
    for i in range(len(input_shape)):
        idx_min.append(indxes[i].min())
        idx_max.append(indxes[i].max())

    for i in range(len(input_shape)):
        idx_min[i] = max(idx_min[i] - margin[i], 0)
        idx_max[i] = min(idx_max[i] + margin[i], input_shape[i] - 1)
    return idx_min, idx_max
  
def crop_ND_volume_with_bounding_box(volume, min_idx, max_idx):
    """
    crop/extract a subregion form an nd image.
    """
    dim = len(volume.shape)
    assert(dim >= 2 and dim <= 5)
    if(dim == 2):
      output = volume[np.ix_(range(min_idx[0], max_idx[0] + 1),
                               range(min_idx[1], max_idx[1] + 1))]
    elif(dim == 3):
        output = volume[np.ix_(range(min_idx[0], max_idx[0] + 1),
                               range(min_idx[1], max_idx[1] + 1),
                               range(min_idx[2], max_idx[2] + 1))]
    elif(dim == 4):
        output = volume[np.ix_(range(min_idx[0], max_idx[0] + 1),
                               range(min_idx[1], max_idx[1] + 1),
                               range(min_idx[2], max_idx[2] + 1),
                               range(min_idx[3], max_idx[3] + 1))]
    elif(dim == 5):
        output = volume[np.ix_(range(min_idx[0], max_idx[0] + 1),
                               range(min_idx[1], max_idx[1] + 1),
                               range(min_idx[2], max_idx[2] + 1),
                               range(min_idx[3], max_idx[3] + 1),
                               range(min_idx[4], max_idx[4] + 1))]
    else:
        raise ValueError("the dimension number shoud be 2 to 5")
    return output

def my_resize(img, shape, mode='constant', orig_shape=(155, 240, 240)):
    """
    Wrapper for scipy.ndimage.zoom suited for MRI images.
    """
    assert len(shape) == 3, "Can not have more than 3 dimensions"
    factors = (
        shape[0]/orig_shape[0],
        shape[1]/orig_shape[1], 
        shape[2]/orig_shape[2]
    )
    
    # Resize to the given shape
    return zoom(img, factors, mode=mode)

def get_random_roi_sampling_center(input_shape, output_shape, sample_mode='full', bounding_box = None):
    """
    get a random coordinate representing the center of a roi for sampling
    inputs:
        input_shape: the shape of sampled volume
        output_shape: the desired roi shape
        sample_mode: 'valid': the entire roi should be inside the input volume
                     'full': only the roi centre should be inside the input volume
        bounding_box: the bounding box which the roi center should be limited to
    outputs:
        center: the output center coordinate of a roi
    """
    center = []
    for i in range(len(input_shape)):
        if(sample_mode[i] == 'full'):
            if(bounding_box):
                x0 = bounding_box[i*2]; x1 = bounding_box[i*2 + 1]
            else:
                x0 = 0; x1 = input_shape[i]
        else:
            if(bounding_box):
                x0 = bounding_box[i*2] + int(output_shape[i]/2)   
                x1 = bounding_box[i*2+1] - int(output_shape[i]/2)   
            else:
                x0 = int(output_shape[i]/2)   
                x1 = input_shape[i] - x0
        if(x1 <= x0):
            centeri = int((x0 + x1)/2)
        else:
            centeri = np.random.randint(x0, x1)
        center.append(centeri)
    return center
def extract_roi_from_volume(volume, in_center, output_shape, fill = 'random'):
    """
    extract a roi from a 3d volume
    inputs:
        volume: the input 3D volume
        in_center: the center of the roi
        output_shape: the size of the roi
        fill: 'random' or 'zero', the mode to fill roi region where is outside of the input volume
    outputs:
        output: the roi volume
    """
    input_shape = volume.shape   
    if(fill == 'random'):
        output = np.random.normal(0, 1, size = output_shape)
    else:
        output = np.zeros(output_shape)
    r0max = [int(x/2) for x in output_shape]
    r1max = [output_shape[i] - r0max[i] for i in range(len(r0max))]
    r0 = [min(r0max[i], in_center[i]) for i in range(len(r0max))]
    r1 = [min(r1max[i], input_shape[i] - in_center[i]) for i in range(len(r0max))]
    out_center = r0max

    output[np.ix_(range(out_center[0] - r0[0], out_center[0] + r1[0]),
                  range(out_center[1] - r0[1], out_center[1] + r1[1]),
                  range(out_center[2] - r0[2], out_center[2] + r1[2]))] = \
        volume[np.ix_(range(in_center[0] - r0[0], in_center[0] + r1[0]),
                      range(in_center[1] - r0[1], in_center[1] + r1[1]),
                      range(in_center[2] - r0[2], in_center[2] + r1[2]))]
    return output

# def zero_padding_3d(volume):
#     # x= np.pad(volume, ((0, 0),(12,11),(30,29)), 'constant', constant_values=0)
#     x= np.pad(volume, ((0, 0),(11,11),(29,29)), 'constant', constant_values=0)
#     return x

def preprocess(img):

    """
    pad
    """
    # img = zero_padding_3d(img)
    # img = itensity_normalize_one_volume(img)

    return img   

def preprocess_label(img):

    """
    pad
    """
    # img = zero_padding_3d(img)
        
    """
    Separates out the 1 labels from the segmentation
    """
    img[img>0]=1
    # ms_plk = img == 1
        
    return img

In [ ]:
################################################################################
# Get a list of files for all modalities individually for train
T1 = glob.glob('/gdrive/MyDrive/new_ms_dataset_me_new_rater/new/*/*/preprocessed/*T1.nii.gz')
T1.sort()
T2 = glob.glob('/gdrive/My Drive/new_ms_dataset_me_new_rater/new/*/*/preprocessed/*T2.nii.gz')
T2.sort()
FLAIR= glob.glob('/gdrive/My Drive/new_ms_dataset_me_new_rater/new/*/*/preprocessed/*FLAIR.nii.gz')
FLAIR.sort()
mask= glob.glob('/gdrive/My Drive/new_ms_dataset_me_new_rater/new/*/*/mask/*mask.nii.gz')  # Ground Truth
mask.sort()

In [ ]:
################################################################################
print(np.shape(FLAIR))
print(np.shape(T1))
print(np.shape(T2))
print(np.shape(mask))
print(FLAIR)
print(T1)
print(T2)
print(mask)

(235,)
(235,)
(235,)
(235,)
['/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/001/preprocessed/S_1_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/002/preprocessed/S_2_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/003/preprocessed/S_3_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/004/preprocessed/S_4_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/005/preprocessed/S_5_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/006/preprocessed/S_6_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/007/preprocessed/S_7_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/008/preprocessed/S_8_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/009/preprocessed/S_9_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/010/preprocessed/S_10_FLAIR.nii.gz', '/gdrive/My Drive/new_ms_datas

In [ ]:
################################################################################
"""
Parse all the filenames and create a dictionary for each patient with structure:

 {   'T1': <path to t1 MRI file>,
     'T2': <path to t2 MRI>,
     'FLAIR': <path to FLAIR MRI file>,
     'mask': <path to Ground Truth file>,
 }
"""
pat = re.compile('.*_(\w*)\.nii\.gz')

data_paths = [{
    pat.findall(item)[0]:item
    for item in items
}
for items in list(zip(T1, T2, FLAIR, mask))]
print(data_paths)
print(type(data_paths))
print(np.shape(data_paths))

[{'T1': '/gdrive/MyDrive/new_ms_dataset_me_new_rater/new/DataSet/001/preprocessed/S_1_T1.nii.gz', 'T2': '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/001/preprocessed/S_1_T2.nii.gz', 'FLAIR': '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/001/preprocessed/S_1_FLAIR.nii.gz', 'mask': '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/001/mask/S_1_mask.nii.gz'}, {'T1': '/gdrive/MyDrive/new_ms_dataset_me_new_rater/new/DataSet/002/preprocessed/S_2_T1.nii.gz', 'T2': '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/002/preprocessed/S_2_T2.nii.gz', 'FLAIR': '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/002/preprocessed/S_2_FLAIR.nii.gz', 'mask': '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/002/mask/S_2_mask.nii.gz'}, {'T1': '/gdrive/MyDrive/new_ms_dataset_me_new_rater/new/DataSet/003/preprocessed/S_3_T1.nii.gz', 'T2': '/gdrive/My Drive/new_ms_dataset_me_new_rater/new/DataSet/003/preprocessed/S_3_T2.nii.gz', 'FLAIR': '/gdrive/My Dr

In [ ]:
################################################################################
def crop_ND_volume_with_bounding_box(volume, min_idx, max_idx):
    """
    crop/extract a subregion form an nd image.
    """
    dim = len(volume.shape)
    channel_dim = volume.shape[-1]
    output = volume[np.ix_(range(min_idx[0], max_idx[0] + 1),
                           range(min_idx[1], max_idx[1] + 1),
                           range(min_idx[2], max_idx[2] + 1),
                           range(channel_dim))]
    return output
def get_random_roi_sampling_center_v2(croped_zero,croped_zero_mask,nonzero_center='both',mask_num=0):
    """
    get a random coordinate representing the center of a roi for sampling
    two option is available:
    1) get roi center that centered on nonzero voxels of mask.
    2) get roi center that centered on nonzero voxels of brain region with a
    margin of border to include more brain region
    """
    if nonzero_center=='both':
       nonzero_center = random.choice(['true','false'])
    if nonzero_center=='true':
       mask_nonzero = np.array(croped_zero_mask[:,:,:,mask_num].nonzero())
       random_select = np.random.randint(0,mask_nonzero.shape[1])
       roi_index = mask_nonzero[:,random_select]
    if nonzero_center=='false':
        nonzero = np.array(croped_zero[:,:,:, 0].nonzero())
        random_select = np.random.randint(0,nonzero.shape[1])
        roi_index = nonzero[:,random_select]
    return roi_index
def extract_roi_from_volume(volume, in_center, output_shape, fill = 'random'):
    """
    extract a roi from a 3d volume
    inputs:
        volume: the input 3D volume
        in_center: the center of the roi
        output_shape: the size of the roi
        fill: 'random' or 'zero', the mode to fill roi region where is outside of the input volume
    outputs:
        output: the roi volume
    """
    input_shape = volume.shape   
    if(fill == 'random'):
        output = np.random.normal(0, 1, size = output_shape)
    else:
        output = np.zeros(output_shape)
    r0max = [int(x/2) for x in output_shape]
    r1max = [output_shape[i] - r0max[i] for i in range(len(r0max))]
    r0 = [min(r0max[i], in_center[i]) for i in range(len(r0max))]
    r1 = [min(r1max[i], input_shape[i] - in_center[i]) for i in range(len(r0max))]
    out_center = r0max

    output[np.ix_(range(out_center[0] - r0[0], out_center[0] + r1[0]),
                  range(out_center[1] - r0[1], out_center[1] + r1[1]),
                  range(out_center[2] - r0[2], out_center[2] + r1[2]))] = \
        volume[np.ix_(range(in_center[0] - r0[0], in_center[0] + r1[0]),
                      range(in_center[1] - r0[1], in_center[1] + r1[1]),
                      range(in_center[2] - r0[2], in_center[2] + r1[2]))]
    return output
def batch(iterable, n=1):
            l = len(iterable)
            for ndx in range(0, l, n):
               yield iterable[ndx:min(ndx + n, l)]
def trainset_generator_3D_ISBI_aug_random_patch_equal_prob(batch_size,num_fold):
        hf_x = h5py.File('/gdrive/My Drive/data_train_3D_me_235sample_new_rater.hdf5', 'r')
        hf_y = h5py.File('/gdrive/My Drive/label_train_3D_me_235sample_new_rater.hdf5', 'r')
        data_len = hf_x['data_train_3D_me_235sample_new_rater.hdf5'].shape[0]
        index = list(range(data_len))
        train_indices=[None]*5
        test_indices=[None]*5
        kf = KFold(n_splits=5,random_state=1,shuffle=True)
        for i, [train_index, test_index] in enumerate(kf.split(index)):
            train_indices[i], test_indices[i] = train_index, test_index
        while True:
            ind = shuffle(train_indices[num_fold])
            for x in batch(ind, batch_size):
                data_x=[]
                data_y=[]
                for i in x:
                    idx_min, idx_max=get_none_zero_region(hf_x['data_train_3D_me_235sample_new_rater.hdf5'][i,:,:,:,0],0)
                    croped_zero = crop_ND_volume_with_bounding_box(hf_x['data_train_3D_me_235sample_new_rater.hdf5'][i,:,:,:,:], idx_min,idx_max) 
                    croped_zero_mask =  crop_ND_volume_with_bounding_box(hf_y['label_train_3D_me_235sample_new_rater.hdf5'][i,:,:,:,:], idx_min,idx_max)
                    roi_sampled = get_random_roi_sampling_center_v2(croped_zero,croped_zero_mask,nonzero_center='both',mask_num=0)
                    norm_input=[]
                    extracted_roi_mask=[] 
                    for w in range(3):
                        extracted_roi = extract_roi_from_volume(croped_zero[:,:,:,w],roi_sampled, (64,64,64), fill = 'zero')
                        norm_modal = itensity_normalize_one_volume(extracted_roi)
                        norm_input.append(norm_modal)  
                    for ww in range(1):
                        extracted_roi_mask.append(extract_roi_from_volume(croped_zero_mask[:,:,:,ww],roi_sampled, (64,64,64), fill = 'zero'))
                    norm_input = np.einsum('CDHW->HWDC', norm_input)
                    extract_roi_mask= np.einsum('CDHW->HWDC', extracted_roi_mask)
                    xy = tf.concat([norm_input, extract_roi_mask], axis=-1)
                    ###fliplr
                    xy = tf.cond(tf.random.uniform(()) > 0.5,lambda: tf.reverse(xy, axis=[1]),lambda: xy)
                    ###flipud
                    xy = tf.cond(tf.random.uniform(()) > 0.5,lambda: tf.reverse(xy, axis=[0]),lambda: xy)
                    # ###rot90 counter-clockwise
                    # xy = tf.cond(tf.random.uniform(()) > 0.5,lambda: np.rot90(xy,axes=(0,1)),lambda: xy)
                    # ###rot90 clockwise
                    # xy = tf.cond(tf.random.uniform(()) > 0.5,lambda: np.rot90(xy,axes=(1,0)),lambda: xy)
                    images_aug, segmaps_aug = tf.split(xy, [3, 1], axis=-1)
                    data_x.append(images_aug.numpy())
                    data_y.append(segmaps_aug.numpy())
                yield(np.array(data_x),np.array(data_y))

In [ ]:
################################################################################
### new efficient generator
def crop_ND_volume_with_bounding_box(volume, min_idx, max_idx):
    """
    crop/extract a subregion form an nd image.
    """
    dim = len(volume.shape)
    channel_dim = volume.shape[-1]
    output = volume[np.ix_(range(min_idx[0], max_idx[0] + 1),
                           range(min_idx[1], max_idx[1] + 1),
                           range(min_idx[2], max_idx[2] + 1))]
    return output
def get_random_roi_sampling_center_v2(croped_zero,croped_zero_mask,nonzero_center='both'):
    """
    get a random coordinate representing the center of a roi for sampling
    two option is available:
    1) get roi center that centered on nonzero voxels of mask.
    2) get roi center that centered on nonzero voxels of brain region with a
    margin of border to include more brain region
    """
    if nonzero_center=='both':
       nonzero_center = random.choice(['true','false'])
    if nonzero_center=='true':
       mask_nonzero = np.array(croped_zero_mask.nonzero())
       random_select = np.random.randint(0,mask_nonzero.shape[1])
       roi_index = mask_nonzero[:,random_select]
    if nonzero_center=='false':
        nonzero = np.array(croped_zero.nonzero())
        random_select = np.random.randint(0,nonzero.shape[1])
        roi_index = nonzero[:,random_select]
    return roi_index
def extract_roi_from_volume(volume, in_center, output_shape, fill = 'random'):
    """
    extract a roi from a 3d volume
    inputs:
        volume: the input 3D volume
        in_center: the center of the roi
        output_shape: the size of the roi
        fill: 'random' or 'zero', the mode to fill roi region where is outside of the input volume
    outputs:
        output: the roi volume
    """
    input_shape = volume.shape   
    if(fill == 'random'):
        output = np.random.normal(0, 1, size = output_shape)
    else:
        output = np.zeros(output_shape)
    r0max = [int(x/2) for x in output_shape]
    r1max = [output_shape[i] - r0max[i] for i in range(len(r0max))]
    r0 = [min(r0max[i], in_center[i]) for i in range(len(r0max))]
    r1 = [min(r1max[i], input_shape[i] - in_center[i]) for i in range(len(r0max))]
    out_center = r0max

    output[np.ix_(range(out_center[0] - r0[0], out_center[0] + r1[0]),
                  range(out_center[1] - r0[1], out_center[1] + r1[1]),
                  range(out_center[2] - r0[2], out_center[2] + r1[2]))] = \
        volume[np.ix_(range(in_center[0] - r0[0], in_center[0] + r1[0]),
                      range(in_center[1] - r0[1], in_center[1] + r1[1]),
                      range(in_center[2] - r0[2], in_center[2] + r1[2]))]
    return output
def get_none_zero_region(im, margin):
    """
    get the bounding box of the non-zero region of an ND volume.
    """
    input_shape = im.shape
    if(type(margin) is int ):
        margin = [margin]*len(input_shape)
    assert(len(input_shape) == len(margin))
    indxes = np.nonzero(im)
    idx_min = []
    idx_max = []
    for i in range(len(input_shape)):
        idx_min.append(indxes[i].min())
        idx_max.append(indxes[i].max())

    for i in range(len(input_shape)):
        idx_min[i] = max(idx_min[i] - margin[i], 0)
        idx_max[i] = min(idx_max[i] + margin[i], input_shape[i] - 1)
    return idx_min, idx_max
def batch(iterable, n=1):
            l = len(iterable)
            for ndx in range(0, l, n):
               yield iterable[ndx:min(ndx + n, l)]

def trainset_generator_3D_ISBI_aug_random_patch_equal_prob(batch_size,num_fold):
        data_len = len(data_paths)
        index = list(range(data_len))
        train_indices=[None]*5
        test_indices=[None]*5
        kf = KFold(n_splits=5,random_state=1,shuffle=True)
        for i, [train_index, test_index] in enumerate(kf.split(index)):
            train_indices[i], test_indices[i] = train_index, test_index
        while True:
            ind = shuffle(train_indices[num_fold])
            for x in batch(ind, batch_size):
                data_x=[]
                data_y=[]
                for i in x:
                    fl = read_img(data_paths[i]['FLAIR'])
                    ms = preprocess_label(read_img(data_paths[i]['mask']))
                    idx_min, idx_max=get_none_zero_region(fl,0)
                    croped_zero = crop_ND_volume_with_bounding_box(fl, idx_min,idx_max) 
                    croped_zero_mask =  crop_ND_volume_with_bounding_box(ms, idx_min,idx_max)
                    roi_sampled = get_random_roi_sampling_center_v2(croped_zero,croped_zero_mask,nonzero_center='both')
                    norm_input=[] 
                    for w in ['FLAIR','T2','T1']:
                        croped_zero = crop_ND_volume_with_bounding_box(read_img(data_paths[i][w]), idx_min,idx_max)
                        extracted_roi = extract_roi_from_volume(croped_zero,roi_sampled, (64,64,64), fill = 'zero')
                        norm_modal = itensity_normalize_one_volume(extracted_roi)
                        norm_input.append(norm_modal)
                    extracted_roi_mask = extract_roi_from_volume(croped_zero_mask,roi_sampled, (64,64,64), fill = 'zero')
                    extracted_roi_mask = np.expand_dims(extracted_roi_mask,axis=0)
                    norm_input = np.einsum('CDHW->HWDC', norm_input)
                    extracted_roi_mask= np.einsum('CDHW->HWDC', extracted_roi_mask)
                    xy = tf.concat([norm_input, extracted_roi_mask], axis=-1)
                    xy = tf.concat([norm_input, extracted_roi_mask], axis=-1)
                    xy = tf.cond(tf.random.uniform(()) > 0.5 ,lambda: tf.reverse(xy, axis=[1]),lambda: xy)
                    xy = tf.cond(tf.random.uniform(()) > 0.5 ,lambda: tf.reverse(xy, axis=[0]),lambda: xy)
                    images_aug, segmaps_aug = tf.split(xy, [3, 1], axis=-1)
                    data_x.append(np.float32(images_aug.numpy()))
                    data_y.append(np.float32(segmaps_aug.numpy()))
                yield(np.array(data_x),np.array(data_y))

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, add, AveragePooling2D, UpSampling2D, SeparableConvolution2D, Dropout, multiply, GlobalAveragePooling2D, Conv3D, Conv3DTranspose, MaxPooling3D, GlobalAveragePooling3D, UpSampling3D
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.optimizers import Adam, SGD, Nadam, Adagrad, RMSprop
from tensorflow.keras.activations import elu, sigmoid
from tensorflow.keras.layers import LeakyReLU, Reshape, Dense, Multiply
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, CSVLogger
from tensorflow.keras.losses import BinaryCrossentropy, mean_squared_error
from tensorflow.keras.metrics import AUC
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Lambda
input_shape = (64,64,64,3)
input_shape_test= (192,192,192,3)
seed_number = 1234
l2 = l2(0.0001)
ki = glorot_uniform(seed=seed_number)
def recall_m(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        true_positives = K.sum(K.round(K.clip(y_true_f* y_pred_f, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_f, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        true_positives = K.sum(K.round(K.clip(y_true_f * y_pred_f, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_f, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def jaccard_distance(y_true, y_pred, smooth=1):
    """Jaccard distance for semantic segmentation.
    Also known as the intersection-over-union loss.
    This loss is useful when you have unbalanced numbers of pixels within an image
    because it gives all classes equal weight. However, it is not the defacto
    standard for image segmentation.
    For example, assume you are trying to predict if
    each pixel is cat, dog, or background.
    You have 80% background pixels, 10% dog, and 10% cat.
    If the model predicts 100% background
    should it be be 80% right (as with categorical cross entropy)
    or 30% (with this loss)?
    The loss has been modified to have a smooth gradient as it converges on zero.
    This has been shifted so it converges on 0 and is smoothed to avoid exploding
    or disappearing gradient.
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    # Arguments
        y_true: The ground truth tensor.
        y_pred: The predicted tensor
        smooth: Smoothing factor. Default is 100.
    # Returns
        The Jaccard distance between the two tensors.
    # References
        - [What is a good evaluation measure for semantic segmentation?](
           http://www.bmva.org/bmvc/2013/Papers/paper0032/paper0032.pdf)
    """
    threshold = 0.5
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_pred_f = K.cast(K.greater_equal(K.clip(y_pred_f, 0, 1), threshold),'float32')
    intersection = K.sum(K.abs(y_true_f * y_pred_f), axis=-1)
    sum_ = K.sum(K.abs(y_true_f) + K.abs(y_pred_f), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return jac
def dice_coef(y_true, y_pred):
    threshold = 0.5
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_pred_f = K.cast(K.greater_equal(K.clip(y_pred_f, 0, 1), threshold),'float32')
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
def dice_coefficient(y_true, y_pred):
    smooth=1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(K.abs(y_true_f * y_pred_f), axis=-1)
    return (2. * intersection + K.epsilon()) / (
        K.sum(K.square(y_true_f), -1) + K.sum(K.square(y_pred_f), -1) + K.epsilon())
def dice_coefficient_correct(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(K.abs(y_true_f * y_pred_f), axis=-1)
    return (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
def confusion(y_true, y_pred):
    smooth=1
    y_pred_f = K.flatten(y_pred)
    y_pred_pos = K.clip(y_pred_f, 0, 1)
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.clip(y_true, 0, 1)
    y_neg = 1 - y_pos
    tp = K.sum(y_pos * y_pred_pos)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg) 
    prec = (tp + smooth)/(tp+fp+smooth)
    recall = (tp+smooth)/(tp+fn+smooth)
    return prec, recall

def tp(y_true, y_pred):
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_pred_pos = K.round(K.clip(y_pred_f, 0, 1))
    y_pos = K.round(K.clip(y_true_f, 0, 1))
    tp = (K.sum(y_pos * y_pred_pos) + smooth)/ (K.sum(y_pos) + smooth) 
    return tp 

def tn(y_true, y_pred):
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_pred_pos = K.round(K.clip(y_pred_f, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true_f, 0, 1))
    y_neg = 1 - y_pos 
    tn = (K.sum(y_neg * y_pred_neg) + smooth) / (K.sum(y_neg) + smooth )
    return tn 

def tversky(y_true, y_pred):
    smooth = 1
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)

def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)
def focal_l2(inputs,output, e=1e-8, weight_L2=0.1):
    """
    loss(input_shape, inp, e=1e-8, weight_L2=0.1)
    ------------------------------------------------------
    Since keras does not allow custom loss functions to have arguments
    other than the true and predicted labels, this function acts as a wrapper
    that allows us to implement the custom loss used in the paper, involving
    outputs from multiple layers.
    L = - L<dice> + weight_L2 ∗ L<L2>
    - L<dice> is the dice loss between input and segmentation output.
    - L<L2> is the L2 loss between the output and the input.

    Parameters
    ----------
    `input_shape`: A 4-tuple, required
        The shape of an image as the tuple (c, H, W, D), where c is
        the no. of channels; H, W and D is the height, width and depth of the
        input image, respectively.
    `inp`: An keras.layers.Layer instance, required
    `e`: Float, optional
        A small epsilon term to add in the denominator to avoid dividing by
        zero and possible gradient explosion.
    `weight_L2`: A real number, optional
        The weight to be given to the L2 loss term in the loss function. Adjust to get best
        results for your task.

    -------
    loss_(y_true, y_pred): A custom keras loss function
        This function takes as input the predicted and ground labels, uses them
        to calculate the dice loss. Combined with the L<L2 computed
        earlier, it returns the total loss.
    """

    loss_L2 = K.mean(K.square(inputs - output), axis=(1, 2, 3))
    
    def loss_(y_true,y_pred):
        pt_1 = tversky(y_true, y_pred)
        gamma = 0.75
        return K.pow((1-pt_1), gamma) + weight_L2 * loss_L2 

    return loss_
def dice_bce_l2(inputs,output, e=1e-8, weight_L2=0.1):
    """
    loss(input_shape, inp, e=1e-8, weight_L2=0.1)
    ------------------------------------------------------
    Since keras does not allow custom loss functions to have arguments
    other than the true and predicted labels, this function acts as a wrapper
    that allows us to implement the custom loss used in the paper, involving
    outputs from multiple layers.
    L = - L<dice> + weight_L2 ∗ L<L2>
    - L<dice> is the dice loss between input and segmentation output.
    - L<L2> is the L2 loss between the output and the input.

    Parameters
    ----------
    `input_shape`: A 4-tuple, required
        The shape of an image as the tuple (c, H, W, D), where c is
        the no. of channels; H, W and D is the height, width and depth of the
        input image, respectively.
    `inp`: An keras.layers.Layer instance, required
    `e`: Float, optional
        A small epsilon term to add in the denominator to avoid dividing by
        zero and possible gradient explosion.
    `weight_L2`: A real number, optional
        The weight to be given to the L2 loss term in the loss function. Adjust to get best
        results for your task.

    -------
    loss_(y_true, y_pred): A custom keras loss function
        This function takes as input the predicted and ground labels, uses them
        to calculate the dice loss. Combined with the L<L2 computed
        earlier, it returns the total loss.
    """

    loss_L2 = K.mean(K.square(inputs - output), axis=(1, 2, 3))
    
    def loss(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(K.abs(y_true_f * y_pred_f), axis=-1)
        loss_dice = (2. * intersection) / (
            K.sum(K.square(y_true_f), -1) + K.sum(K.square(y_pred_f), -1) + e)

        return 1- loss_dice + weight_L2 * loss_L2 + binary_crossentropy(y_true_f, y_pred_f)

    return loss
def dice_l2(inputs,output, e=1e-8, weight_L2=0.1):
    loss_L2 = K.mean(K.square(inputs - output), axis=(1, 2, 3))
    
    def loss(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(K.abs(y_true_f * y_pred_f), axis=-1)
        loss_dice = (2. * intersection) / (
        K.sum(K.square(y_true_f), -1) + K.sum(K.square(y_pred_f), -1) + e)

        return 1- loss_dice + weight_L2 * loss_L2

    return loss
################################################################################
######model_1: MultiRes_Unet


def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), activation='relu', name=None):
    output_channels=1
    input_shape=(120,120,2)
    '''
    2D Convolutional layers
    
    Arguments:
        x {keras layer} -- input layer 
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters
    
    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(1, 1)})
        activation {str} -- activation function (default: {'relu'})
        name {str} -- name of the layer (default: {None})
    
    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2D(filters, (num_row, num_col), strides=strides, padding=padding, use_bias=False)(x)
    x = BatchNormalization(axis=3, scale=False)(x)

    if(activation == None):
        return x

    x = Activation(activation, name=name)(x)

    return x


def trans_conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(2, 2), name=None):
    '''
    2D Transposed Convolutional layers
    
    Arguments:
        x {keras layer} -- input layer 
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters
    
    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(2, 2)})
        name {str} -- name of the layer (default: {None})
    
    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2DTranspose(filters, (num_row, num_col), strides=strides, padding=padding)(x)
    x = BatchNormalization(axis=3, scale=False)(x)
    
    return x


def MultiResBlock(U, inp, alpha = 1.67):
    '''
    MultiRes Block
    
    Arguments:
        U {int} -- Number of filters in a corrsponding UNet stage
        inp {keras layer} -- input layer 
    
    Returns:
        [keras layer] -- [output layer]
    '''

    W = alpha * U

    shortcut = inp

    shortcut = conv2d_bn(shortcut, int(W*0.167) + int(W*0.333) +
                         int(W*0.5), 1, 1, activation=None, padding='same')

    conv3x3 = conv2d_bn(inp, int(W*0.167), 3, 3,
                        activation='relu', padding='same')

    conv5x5 = conv2d_bn(conv3x3, int(W*0.333), 3, 3,
                        activation='relu', padding='same')

    conv7x7 = conv2d_bn(conv5x5, int(W*0.5), 3, 3,
                        activation='relu', padding='same')

    out = concatenate([conv3x3, conv5x5, conv7x7], axis=3)
    out = BatchNormalization(axis=3)(out)

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    return out


def ResPath(filters, length, inp):
    '''
    ResPath
    
    Arguments:
        filters {int} -- [description]
        length {int} -- length of ResPath
        inp {keras layer} -- input layer 
    
    Returns:
        [keras layer] -- [output layer]
    '''


    shortcut = inp
    shortcut = conv2d_bn(shortcut, filters, 1, 1,
                         activation=None, padding='same')

    out = conv2d_bn(inp, filters, 3, 3, activation='relu', padding='same')

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    for i in range(length-1):

        shortcut = out
        shortcut = conv2d_bn(shortcut, filters, 1, 1,
                             activation=None, padding='same')

        out = conv2d_bn(out, filters, 3, 3, activation='relu', padding='same')

        out = add([shortcut, out])
        out = Activation('relu')(out)
        out = BatchNormalization(axis=3)(out)

    return out


def MultiResUnet(height, width, n_channels, output_channels=1, lr=0.0001):
    '''
    MultiResUNet
    
    Arguments:
        height {int} -- height of image 
        width {int} -- width of image 
        n_channels {int} -- number of channels in image
    
    Returns:
        [keras model] -- MultiResUNet model
    '''


    inputs = Input((height, width, n_channels))

    mresblock1 = MultiResBlock(16, inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(mresblock1)
    mresblock1 = ResPath(16, 4, mresblock1)

    mresblock2 = MultiResBlock(16*2, pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(mresblock2)
    mresblock2 = ResPath(16*2, 3, mresblock2)

    mresblock3 = MultiResBlock(16*4, pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(mresblock3)
    mresblock3 = ResPath(16*4, 2, mresblock3)

    mresblock4 = MultiResBlock(16*8, pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(mresblock4)
    mresblock4 = ResPath(16*8, 1, mresblock4)

    mresblock5 = MultiResBlock(16*16, pool4)

    up6 = concatenate([Conv2DTranspose(
        16*8, (2, 2), strides=(2, 2), padding='same')(mresblock5), mresblock4], axis=3)
    mresblock6 = MultiResBlock(16*8, up6)

    up7 = concatenate([Conv2DTranspose(
        16*4, (2, 2), strides=(2, 2), padding='same')(mresblock6), mresblock3], axis=3)
    mresblock7 = MultiResBlock(16*4, up7)

    up8 = concatenate([Conv2DTranspose(
        16*2, (2, 2), strides=(2, 2), padding='same')(mresblock7), mresblock2], axis=3)
    mresblock8 = MultiResBlock(16*2, up8)

    up9 = concatenate([Conv2DTranspose(16, (2, 2), strides=(
        2, 2), padding='same')(mresblock8), mresblock1], axis=3)
    mresblock9 = MultiResBlock(16, up9)

    conv10 = conv2d_bn(mresblock9, 1, 1, 1, activation='relu')
    conv11 =Conv2D(filters=output_channels,kernel_size=(1, 1),strides=1,activation='sigmoid')(conv10)
    
    model = Model(inputs=[inputs], outputs=[conv11])
    
    output=conv10
    model.compile(Adam(lr=lr),loss=focal_tversky,
    metrics=[dice_coef,dice_coefficient,jaccard_distance,AUC(num_thresholds=500)])

    return model
################################################################################
######model_2: mini_2D_Unet

def mini_2D_Unet(input_shape, cout=1, final_activ='sigmoid',opt='SGD', lr=0.0001, weight_L2=0.001):
    input_img =Input(shape=input_shape)
    x = Conv2D(16, (3,3), activation='relu', padding='same', name='c1_1',data_format='channels_last')(input_img)  
    x = Conv2D(16, (3,3), activation='relu', padding='same', name='c1_2',data_format='channels_last')(x)   
    c1= Conv2D(32, (3,3), activation='relu', padding='same', name='c1_3',data_format='channels_last')(x)   
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='c1_4', strides=2,data_format='channels_last')(c1)     
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='c2_1',data_format='channels_last')(x)   
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='c2_2',data_format='channels_last')(x)    
    c2= Conv2D(64, (3,3), activation='relu', padding='same', name='c2_3',data_format='channels_last')(x) 
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='c2_4', strides=2,data_format='channels_last')(c2)    
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='c3_1',data_format='channels_last')(x)  
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='c3_2',data_format='channels_last')(x)   
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='c3_3',data_format='channels_last')(x)   
    x = Conv2DTranspose(64, (3,3), activation='relu', padding='same', name='d1', strides=2,data_format='channels_last')(x)   
    x = concatenate([x, c2], axis=-1, name='m1') 
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='c4_1',data_format='channels_last')(x)   
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='c4_2',data_format='channels_last')(x)     
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='c4_3',data_format='channels_last')(x)     
    x = Conv2DTranspose(32, (3,3), activation='relu', padding='same', name='d2', strides=2,data_format='channels_last')(x)    
    x = concatenate([x, c1],axis=-1, name='m2')
    x = Conv2D(16, (3,3), activation='relu', padding='same', name='c5_1',data_format='channels_last')(x)   
    x = Conv2D(16, (3,3), activation='relu', padding='same', name='c5_2',data_format='channels_last')(x)    
    x = Conv2D(16, (3,3), activation='relu', padding='same', name='c5_3',data_format='channels_last')(x)      
    x = Conv2D(cout, (3,3), activation=final_activ, padding='same', name='cout',data_format='channels_last')(x)


    model = Model(inputs=[input_img], outputs=[x], name='mini_unet')
    output= x
    model.compile(adam(lr=lr),loss=loss(input_shape, input_img, output, weight_L2=weight_L2),
    metrics=[dice_coefficient, jaccard_distance])

    return model
################################################################################
######model_3: NoDownSample_net
def NoDownSample_net(input_shape, cout=1, final_activ='sigmoid', opt='Adam', lr=1., weight_L2=0.001):
    def residual_unit(x, filters, size, name=''):  
        f1, f2 = filters  
        c = Conv2D(f1, (size,size), padding='same', name=name+'_c1')(x)  
        c = BatchNormalization(name=name+'_bn1')(c)   
        c = Activation('relu', name=name+'_r1')(c)

        c = Conv2D(f2, (1,1), padding='same', name=name+'_c2')(x)  
        c = BatchNormalization(name=name+'_bn2')(c)   
        c = Activation('relu', name=name+'_r2')(c) 

        c = Add(name=name+'_ad')([c, x])  
        c = Activation('relu', name=name+'_r3')(c)
        return c

    input_img = Input(shape=input_shape, name='input')    
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='c1')(input_img)    
    x = residual_unit(x, (64, 32), 3, name='c1_1')       
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='c2')(x)   
    x = residual_unit(x, (128, 64), 3, name='c2_1')     
    x = residual_unit(x, (128, 64), 3, name='c2_2')   
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='c3')(x) 
    x = residual_unit(x, (64, 32), 3, name='c3_1')   
    x = Conv2D(cout, (3,3), activation=final_activ, padding='same', name='cout')(x) 
    model = Model(input_img, x, name='NoDownSample_net')
    model.compile(Adam(lr=lr),loss=focal_tversky,
    metrics=[dice_coef, jaccard_distance])
    return model
################################################################################
######model_4:densenet
def denseNet(input_shape, final_activ='sigmoid', opt='Adam', cout=1, lr=0.0001, nb_dense_block=2, growth_rate=2, nb_filter=8, reduction=0.5,
             dropout_rate=0.0, weight_decay=1e-4, weights_path=None, weight_L2=0):
    '''Instantiate the DenseNet 121 architecture,
        # Arguments
            nb_dense_block: number of dense blocks to add to end
            growth_rate: number of filters to add per dense block
            nb_filter: initial number of filters
            reduction: reduction factor of transition blocks.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            classes: optional number of classes to classify images
            weights_path: path to pre-trained weights
        # Returns
            A Keras model instance.
    '''
    eps = 1.1e-5

    def conv_block(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
        '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout
            # Arguments
                x: input tensor 
                stage: index for dense block
                branch: layer index within each dense block
                nb_filter: number of filters
                dropout_rate: dropout rate
                weight_decay: weight decay factor
        '''
        conv_name_base = 'conv' + str(stage) + '_' + str(branch)
        relu_name_base = 'relu' + str(stage) + '_' + str(branch)
        # 1x1 Convolution (Bottleneck layer)
        inter_channel = nb_filter * 4  
        x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_x1_bn')(x) 
        x = Activation('relu', name=relu_name_base+'_x1')(x)
        x = Conv2D(inter_channel, 1, padding='same', name=conv_name_base+'_x1', use_bias=False)(x)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)
        # 3x3 Convolution
        x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_x2_bn')(x) 
        x = Activation('relu', name=relu_name_base+'_x2')(x) 
        x = Conv2D(nb_filter, 3, padding='same', name=conv_name_base+'_x2', use_bias=False)(x)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)
        return x
    
    def transition_block(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
        ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout 
            # Arguments
                x: input tensor
                stage: index for dense block
                nb_filter: number of filters
                compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
                dropout_rate: dropout rate
                weight_decay: weight decay factor
        '''
        conv_name_base = 'conv' + str(stage) + '_blk'
        relu_name_base = 'relu' + str(stage) + '_blk'
        pool_name_base = 'pool' + str(stage) 
        x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_bn')(x) 
        x = Activation('relu', name=relu_name_base)(x)
        x = Conv2D(int(nb_filter * compression), 1, padding='same', name=conv_name_base, use_bias=False)(x)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)
        x = AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base)(x)
        return x
    
    def up_transition_block(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
        ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout 
            # Arguments
                x: input tensor
                stage: index for dense block
                nb_filter: number of filters
                compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
                dropout_rate: dropout rate
                weight_decay: weight decay factor
        '''
        conv_name_base = 'conv' + str(stage) + '_blk'
        relu_name_base = 'relu' + str(stage) + '_blk'
        pool_name_base = 'ups' + str(stage) 
        x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base+'_bn')(x) 
        x = Activation('relu', name=relu_name_base)(x)
        x = Conv2D(int(nb_filter * compression), 1, padding='same', name=conv_name_base, use_bias=False)(x)
        if dropout_rate:
            x = Dropout(dropout_rate)(x)  
        x = UpSampling2D(size=(2,2), name=pool_name_base)(x) 
        return x


    def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, grow_nb_filters=True):
        ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
            # Arguments
                x: input tensor
                stage: index for dense block
                nb_layers: the number of layers of conv_block to append to the model.
                nb_filter: number of filters
                growth_rate: growth rate
                dropout_rate: dropout rate
                weight_decay: weight decay factor
                grow_nb_filters: flag to decide to allow number of filters to grow
        '''
        concat_feat = x
        for i in range(nb_layers):
            branch = i+1
            x = conv_block(concat_feat, stage, branch, nb_filter, dropout_rate, weight_decay)
            concat_feat = concatenate([concat_feat, x], axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
            if grow_nb_filters:
                nb_filter += growth_rate
        return concat_feat, nb_filter
    

    # compute compression factor
    compression = 1.0 - reduction

    # Handle Dimension Ordering for different backends
    concat_axis = 3
    img_input = Input(shape=input_shape, name='data') 
    
    # From architecture for ImageNet (Table 1 in the paper)
    #nb_filter = 64
    nb_layers = [6,12,6] # For DenseNet-121

    # Initial convolution 
    x = Conv2D(nb_filter, 7, padding='same',  name='conv1', use_bias=False)(img_input)
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv1_bn')(x) 
    x = Activation('relu', name='relu1')(x)
    
    # Add dense blocks
    for block_idx in range(nb_dense_block):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, 
                                   growth_rate, dropout_rate=dropout_rate, 
                                   weight_decay=weight_decay) 
        # Add transition_block
        x = transition_block(x, stage, nb_filter, compression=compression, 
                             dropout_rate=dropout_rate, weight_decay=weight_decay) 
        nb_filter = int(nb_filter * compression)

     
    for block_idx in range(nb_dense_block ):    
        up_stage = block_idx + stage + 2
        x, nb_filter = dense_block(x, up_stage, nb_layers[block_idx], nb_filter, 
                                   growth_rate, dropout_rate=dropout_rate, 
                                   weight_decay=weight_decay) 
        # Add up_transition_block
        x = up_transition_block(x, up_stage, nb_filter, compression=compression, 
                                dropout_rate=dropout_rate, weight_decay=weight_decay) 
        nb_filter = int(nb_filter * compression)
 
    final_stage = up_stage + 1
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv'+str(final_stage)+'_blk_bn')(x) 
    x = Activation('relu', name='relu'+str(final_stage)+'_blk')(x)
    x = Conv2D(cout, 3, padding="same")(x) 
    x = Activation('sigmoid')(x)
    model = Model(img_input, x, name='densenet')
    model.compile(Adam(lr=lr),loss=focal_tversky,
    metrics=[dice_coef,dice_coefficient,jaccard_distance,AUC(num_thresholds=500)])
    return model

################################################################################
######model_5: ResUnet  
def ResUnet(input_shape, cout=1, final_activ='sigmoid', opt='Adam', lr=0.0001, weight_L2=0):
    def conv_block(input_tensor, num_filters):
        encoder = BatchNormalization()(input_tensor)
        encoder = Activation('relu')(encoder)
        encoder = Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(encoder)
        encoder = BatchNormalization()(encoder)
        encoder = Activation('relu')(encoder)
        encoder = Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(encoder)
        encoder = add([input_tensor,encoder])
        encoder = BatchNormalization()(encoder)
        encoder = Activation('relu')(encoder)
        return encoder 
    def decoder_conv_block(input_tensor, num_filters):
        encoder = BatchNormalization()(input_tensor)
        encoder = Activation('relu')(encoder)
        encoder = Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(encoder)
        encoder = add([input_tensor,encoder])
        encoder = BatchNormalization()(encoder)
        encoder = Activation('relu')(encoder)
        return encoder
        
        

    def encoder_block(input_tensor, num_filters):
        encoder = conv_block(input_tensor, num_filters)
        encoder_pool = MaxPooling2D((2, 2), strides=(2, 2))(encoder)

        return encoder_pool, encoder

    def decoder_block(input_tensor, concat_tensor, num_filters):
        decoder = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(input_tensor)
        decoder = concatenate([concat_tensor, decoder], axis=-1)
        decoder = Conv2D(num_filters, (1, 1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(decoder)
        decoder = decoder_conv_block(decoder, num_filters)
        return decoder 
    inputs = Input(shape=(240,240,2))
    first_conv = Conv2D(32,(3,3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(inputs)
    encoder0_pool, encoder0 = encoder_block(first_conv, 32) 
    encoder0_pool_conv_2D = Conv2D(64,(1,1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(encoder0_pool)
    encoder1_pool, encoder1 = encoder_block(encoder0_pool_conv_2D, 64)  
    encoder1_pool_conv_2D = Conv2D(128,(1,1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(encoder1_pool)
    encoder2_pool, encoder2 = encoder_block(encoder1_pool_conv_2D, 128) 
    encoder2_pool_conv_2D = Conv2D(256,(1,1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(encoder2_pool)
    encoder3_pool, encoder3 = encoder_block(encoder2_pool_conv_2D, 256) 
    encoder3_pool_conv_2D = Conv2D(512,(1,1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(encoder3_pool)
    center = conv_block(encoder3_pool_conv_2D, 512)
    decoder3 = decoder_block(center, encoder3, 256) 
    decoder2 = decoder_block(decoder3, encoder2, 128) 
    decoder1 = decoder_block(decoder2, encoder1, 64) 
    decoder0 = decoder_block(decoder1, encoder0, 32) 
    outputs = Conv2D(cout, (1, 1), activation=final_activ, kernel_initializer=ki, kernel_regularizer=l2)(decoder0) 

    model = Model(inputs=[inputs], outputs=[outputs], name='ResUnet')
    output= outputs
    model.compile(Adam(lr=lr),loss=focal_tversky,
    metrics=[dice_coef,dice_coefficient,jaccard_distance,AUC(num_thresholds=500)])
    return model
#loss(inputs, output, weight_L2=weight_L2)
################################################################################
######model_6: Unet  
def Unet(input_shape, cout=1, final_activ='sigmoid'):
    def conv_block(input_tensor, num_filters):
        encoder = Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(input_tensor)
        encoder = BatchNormalization()(encoder)
        encoder = Activation('relu')(encoder)
        encoder = Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(encoder)
        encoder = BatchNormalization()(encoder)
        encoder = Activation('relu')(encoder)
        return encoder


    def encoder_block(input_tensor, num_filters):
        encoder = conv_block(input_tensor, num_filters)
        encoder_pool = MaxPooling2D((2, 2), strides=(2, 2))(encoder)

        return encoder_pool, encoder

    def decoder_block(input_tensor, concat_tensor, num_filters):
        decoder = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(input_tensor)
        decoder = concatenate([concat_tensor, decoder], axis=-1)
        decoder = BatchNormalization()(decoder)
        decoder = Activation('relu')(decoder)
        decoder = Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(decoder)
        decoder = BatchNormalization()(decoder)
        decoder = Activation('relu')(decoder)
        decoder = Conv2D(num_filters, (3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(decoder)
        decoder = BatchNormalization()(decoder)
        decoder = Activation('relu')(decoder)
        return decoder 
    inputs = Input(shape=(240,240,2)) 
    encoder0_pool, encoder0 = encoder_block(inputs, 128) 
    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 128)  
    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 256) 
    encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) 
    center = conv_block(encoder3_pool, 512) 
    decoder3 = decoder_block(center, encoder3, 256) 
    decoder2 = decoder_block(decoder3, encoder2, 256) 
    decoder1 = decoder_block(decoder2, encoder1, 128) 
    decoder0 = decoder_block(decoder1, encoder0, 128) 
    outputs = Conv2D(cout, (1, 1), activation=final_activ, kernel_initializer=ki, kernel_regularizer=l2)(decoder0) 

    model = Model(inputs=[inputs], outputs=[outputs], name='Unet')
    output= outputs
    model.compile(optimizer=Nadam(lr=0.0001), loss=[focal_tversky], metrics=[dice_coefficient_correct])
    return model
################################################################################
#####model_7: Attention_Unet_version
def expend_as(tensor, rep,name):
	my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep},  name='psi_up'+name)(tensor)
	return my_repeat


def AttnGatingBlock(x, g, inter_shape, name):
    ''' take g which is the spatially smaller signal, do a conv to get the same
    number of feature channels as x (bigger spatially)
    do a conv on x to also get same feature channels (theta_x)
    then, upsample g to be same size as x 
    add x and g (concat_xg)
    relu, 1x1 conv, then sigmoid then upsample the final - this gives us attn coefficients'''
    
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same', name='xl'+name, kernel_initializer=ki, kernel_regularizer=l2)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same', name='g_up'+name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same', name='psi'+name, kernel_initializer=ki, kernel_regularizer=l2)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = expend_as(upsample_psi, shape_x[3],  name)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv2D(shape_x[3], (1, 1), padding='same',name='q_attn_conv'+name, kernel_initializer=ki, kernel_regularizer=l2)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetConv2D(input, outdim, is_batchnorm, name):
	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_1')(input)
	if is_batchnorm:
		x =BatchNormalization(name=name + '_1_bn')(x)
	x = Activation('relu',name=name + '_1_act')(x)

	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_2')(x)
	if is_batchnorm:
		x = BatchNormalization(name=name + '_2_bn')(x)
	x = Activation('relu', name=name + '_2_act')(x)
	return x
	

def UnetGatingSignal(input, is_batchnorm, name):
    ''' this is simply 1x1 convolution, bn, activation '''
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 1, (1, 1), strides=(1, 1), padding="same",  kernel_initializer=ki, kernel_regularizer=l2, name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name = name + '_act')(x)
    return x

# plain old attention gates in u-net, NO multi-input, NO deep supervision
def attn_unet(input_size):   
    inputs = Input(shape=input_size)
    conv1 = UnetConv2D(inputs, 32, is_batchnorm=True, name='conv1')
    # conv1 = Dropout(0.2,name='drop_conv1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = UnetConv2D(pool1, 64, is_batchnorm=True, name='conv2')
    # conv2 = Dropout(0.2,name='drop_conv2')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = UnetConv2D(pool2, 128, is_batchnorm=True, name='conv3')
    # conv3 = Dropout(0.1,name='drop_conv3')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = UnetConv2D(pool3, 256, is_batchnorm=True, name='conv4')
    # conv4 = Dropout(0.1, name='drop_conv4')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    center = UnetConv2D(pool4, 512, is_batchnorm=True, name='center')
    
    g1 = UnetGatingSignal(center, is_batchnorm=True, name='g1')
    attn1 = AttnGatingBlock(conv4, g1, 128, '_1')
    up1 = concatenate([Conv2DTranspose(256, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(center), attn1], name='up1')
    
    g2 = UnetGatingSignal(up1, is_batchnorm=True, name='g2')
    attn2 = AttnGatingBlock(conv3, g2, 64, '_2')
    up2 = concatenate([Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up1), attn2], name='up2')

    g3 = UnetGatingSignal(up1, is_batchnorm=True, name='g3')
    attn3 = AttnGatingBlock(conv2, g3, 32, '_3')
    up3 = concatenate([Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up2), attn3], name='up3')

    up4 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up3), conv1], name='up4')
    out = Conv2D(1, (1, 1), activation='sigmoid',  kernel_initializer=ki, name='final')(up4)
    
    model = Model(inputs=[inputs], outputs=[out])
    model.compile(optimizer=SGD(lr=0.01, momentum=0.9, decay=1e-8), loss=[focal_tversky], metrics=[dice_coef,dice_coefficient,jaccard_distance,AUC(num_thresholds=500)])
    return model
################################################################################
#####model_8: Attention_Unet_deep supervision_version
def expend_as(tensor, rep,name):
	my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep},  name='psi_up'+name)(tensor)
	return my_repeat


def AttnGatingBlock(x, g, inter_shape, name):
    ''' take g which is the spatially smaller signal, do a conv to get the same
    number of feature channels as x (bigger spatially)
    do a conv on x to also get same geature channels (theta_x)
    then, upsample g to be same size as x 
    add x and g (concat_xg)
    relu, 1x1 conv, then sigmoid then upsample the final - this gives us attn coefficients'''
    
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same', name='xl'+name, kernel_initializer=ki, kernel_regularizer=l2)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same', name='g_up'+name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same', name='psi'+name, kernel_initializer=ki, kernel_regularizer=l2)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = expend_as(upsample_psi, shape_x[3],  name)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv2D(shape_x[3], (1, 1), padding='same',name='q_attn_conv'+name, kernel_initializer=ki, kernel_regularizer=l2)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetConv2D(input, outdim, is_batchnorm, name):
	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_1')(input)
	if is_batchnorm:
		x =BatchNormalization(name=name + '_1_bn')(x)
	x = Activation('relu',name=name + '_1_act')(x)

	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_2')(x)
	if is_batchnorm:
		x = BatchNormalization(name=name + '_2_bn')(x)
	x = Activation('relu', name=name + '_2_act')(x)
	return x
	

def UnetGatingSignal(input, is_batchnorm, name):
    ''' this is simply 1x1 convolution, bn, activation '''
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 1, (1, 1), strides=(1, 1), padding="same",  kernel_initializer=ki, kernel_regularizer=l2, name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name = name + '_act')(x)
    return x
#regular attention unet with  deep supervision
def attn_unet_ds(input_size):   
    inputs = Input(shape=input_size)
    conv1 = UnetConv2D(inputs, 32, is_batchnorm=True, name='conv1')
    # conv1 = Dropout(0.2,name='drop_conv1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = UnetConv2D(pool1, 64, is_batchnorm=True, name='conv2')
    # conv2 = Dropout(0.2,name='drop_conv2')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = UnetConv2D(pool2, 128, is_batchnorm=True, name='conv3')
    # conv3 = Dropout(0.1,name='drop_conv3')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = UnetConv2D(pool3, 64, is_batchnorm=True, name='conv4')
    # conv4 = Dropout(0.1, name='drop_conv4')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    center = UnetConv2D(pool4, 512, is_batchnorm=True, name='center')
    
    g1 = UnetGatingSignal(center, is_batchnorm=True, name='g1')
    attn1 = AttnGatingBlock(conv4, g1, 128, '_1')
    up1 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(center), attn1], name='up1')
    
    g2 = UnetGatingSignal(up1, is_batchnorm=True, name='g2')
    attn2 = AttnGatingBlock(conv3, g2, 64, '_2')
    up2 = concatenate([Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up1), attn2], name='up2')

    g3 = UnetGatingSignal(up1, is_batchnorm=True, name='g3')
    attn3 = AttnGatingBlock(conv2, g3, 32, '_3')
    up3 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up2), attn3], name='up3')

    up4 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up3), conv1], name='up4')

    # out = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='final')(up4)
    conv6 = UnetConv2D(up1, 256, is_batchnorm=True, name='conv6')
    conv7 = UnetConv2D(up2, 128, is_batchnorm=True, name='conv7')
    conv8 = UnetConv2D(up3, 64, is_batchnorm=True, name='conv8')
    conv9 = UnetConv2D(up4, 32, is_batchnorm=True, name='conv9')


    out6 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='pred1')(conv6)
    out7 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='pred2')(conv7)
    out8 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='pred3')(conv8)
    out9 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='final')(conv9)


    model = Model(inputs=[inputs], outputs=[out6, out7, out8, out9])
    loss = {'pred1':focal_tversky,
            'pred2':focal_tversky,
            'pred3':focal_tversky,
            'final':focal_tversky}
    
    loss_weights = {'pred1':1,
                    'pred2':1,
                    'pred3':1,
                    'final':1}
    model.compile(optimizer=SGD(lr=0.1, momentum=0.8, nesterov=True), loss=loss, metrics=[dice_coef,jaccard_distance,AUC(num_thresholds=500)])
    return model
################################################################################
#####model_9: Attention_Unet_deep supervision_input pyramid_version
def expend_as(tensor, rep,name):
	my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep},  name='psi_up'+name)(tensor)
	return my_repeat


def AttnGatingBlock(x, g, inter_shape, name):
    ''' take g which is the spatially smaller signal, do a conv to get the same
    number of feature channels as x (bigger spatially)
    do a conv on x to also get same geature channels (theta_x)
    then, upsample g to be same size as x 
    add x and g (concat_xg)
    relu, 1x1 conv, then sigmoid then upsample the final - this gives us attn coefficients'''
    
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same', name='xl'+name, kernel_initializer=ki, kernel_regularizer=l2)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same', name='g_up'+name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same', name='psi'+name, kernel_initializer=ki, kernel_regularizer=l2)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = expend_as(upsample_psi, shape_x[3],  name)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv2D(shape_x[3], (1, 1), padding='same',name='q_attn_conv'+name, kernel_initializer=ki, kernel_regularizer=l2)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetConv2D(input, outdim, is_batchnorm, name):
	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_1')(input)
	if is_batchnorm:
		x =BatchNormalization(name=name + '_1_bn')(x)
	x = Activation('relu',name=name + '_1_act')(x)

	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_2')(x)
	if is_batchnorm:
		x = BatchNormalization(name=name + '_2_bn')(x)
	x = Activation('relu', name=name + '_2_act')(x)
	return x
	

def UnetGatingSignal(input, is_batchnorm, name):
    ''' this is simply 1x1 convolution, bn, activation '''
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 1, (1, 1), strides=(1, 1), padding="same",  kernel_initializer=ki, kernel_regularizer=l2, name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name = name + '_act')(x)
    return x
#regular attention unet with  deep supervision
def attn_unet_ds_ip(input_size):   
    inputs = Input(shape=input_size)
    scale_img_2 = AveragePooling2D(pool_size=(2, 2), name='input_scale2')(inputs)
    scale_img_3 = AveragePooling2D(pool_size=(2, 2), name='input_scale3')(scale_img_2)
    scale_img_4 = AveragePooling2D(pool_size=(2, 2), name='input_scale4')(scale_img_3)


    conv1 = UnetConv2D(inputs, 32, is_batchnorm=True, name='conv1')
    # conv1 = Dropout(0.2,name='drop_conv1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    input2 = Conv2D(64, (3, 3), padding='same', activation='relu', name='conv_scale2', kernel_initializer=ki, kernel_regularizer=l2)(scale_img_2)
    input2 = concatenate([input2, pool1], axis=3)
    conv2 = UnetConv2D(pool1, 64, is_batchnorm=True, name='conv2')
    # conv2 = Dropout(0.2,name='drop_conv2')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    input3 = Conv2D(128, (3, 3), padding='same', activation='relu', name='conv_scale3', kernel_initializer=ki, kernel_regularizer=l2)(scale_img_3)
    input3 = concatenate([input3, pool2], axis=3)
    conv3 = UnetConv2D(pool2, 128, is_batchnorm=True, name='conv3')
    # conv3 = Dropout(0.1,name='drop_conv3')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    input4 = Conv2D(256, (3, 3), padding='same', activation='relu', name='conv_scale4', kernel_initializer=ki, kernel_regularizer=l2)(scale_img_4)
    input4 = concatenate([input4, pool3], axis=3)
    conv4 = UnetConv2D(pool3, 64, is_batchnorm=True, name='conv4')
    # conv4 = Dropout(0.1, name='drop_conv4')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    center = UnetConv2D(pool4, 512, is_batchnorm=True, name='center')
    
    g1 = UnetGatingSignal(center, is_batchnorm=True, name='g1')
    attn1 = AttnGatingBlock(conv4, g1, 128, '_1')
    up1 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(center), attn1], name='up1')
    
    g2 = UnetGatingSignal(up1, is_batchnorm=True, name='g2')
    attn2 = AttnGatingBlock(conv3, g2, 64, '_2')
    up2 = concatenate([Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up1), attn2], name='up2')

    g3 = UnetGatingSignal(up1, is_batchnorm=True, name='g3')
    attn3 = AttnGatingBlock(conv2, g3, 32, '_3')
    up3 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up2), attn3], name='up3')

    up4 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up3), conv1], name='up4')

    conv6 = UnetConv2D(up1, 256, is_batchnorm=True, name='conv6')
    conv7 = UnetConv2D(up2, 128, is_batchnorm=True, name='conv7')
    conv8 = UnetConv2D(up3, 64, is_batchnorm=True, name='conv8')
    conv9 = UnetConv2D(up4, 32, is_batchnorm=True, name='conv9')


    out6 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='pred1')(conv6)
    out7 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='pred2')(conv7)
    out8 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='pred3')(conv8)
    out9 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='final')(conv9)


    model = Model(inputs=[inputs], outputs=[out6, out7, out8, out9])
    loss = {'pred1':focal_tversky,
            'pred2':focal_tversky,
            'pred3':focal_tversky,
            'final':focal_tversky}
    
    loss_weights = {'pred1':1,
                    'pred2':1,
                    'pred3':1,
                    'final':1}
    model.compile(optimizer=SGD(lr=0.1, momentum=0.9, nesterov=True), loss=loss, metrics=[dice_coef,jaccard_distance,AUC(num_thresholds=500)])
    return model
################################################################################
#####model_10: Attention_Unet_deep supervision_psp_version
def expend_as(tensor, rep,name):
	my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep},  name='psi_up'+name)(tensor)
	return my_repeat


def AttnGatingBlock(x, g, inter_shape, name):
    ''' take g which is the spatially smaller signal, do a conv to get the same
    number of feature channels as x (bigger spatially)
    do a conv on x to also get same geature channels (theta_x)
    then, upsample g to be same size as x 
    add x and g (concat_xg)
    relu, 1x1 conv, then sigmoid then upsample the final - this gives us attn coefficients'''
    
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same', name='xl'+name, kernel_initializer=ki, kernel_regularizer=l2)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same', name='g_up'+name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same', name='psi'+name, kernel_initializer=ki, kernel_regularizer=l2)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = expend_as(upsample_psi, shape_x[3],  name)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv2D(shape_x[3], (1, 1), padding='same',name='q_attn_conv'+name, kernel_initializer=ki, kernel_regularizer=l2)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetConv2D(input, outdim, is_batchnorm, name):
	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_1')(input)
	if is_batchnorm:
		x =BatchNormalization(name=name + '_1_bn')(x)
	x = Activation('relu',name=name + '_1_act')(x)

	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_2')(x)
	if is_batchnorm:
		x = BatchNormalization(name=name + '_2_bn')(x)
	x = Activation('relu', name=name + '_2_act')(x)
	return x
	

def UnetGatingSignal(input, is_batchnorm, name):
    ''' this is simply 1x1 convolution, bn, activation '''
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 1, (1, 1), strides=(1, 1), padding="same",  kernel_initializer=ki, kernel_regularizer=l2, name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name = name + '_act')(x)
    return x
#regular attention unet with  deep supervision
def attn_unet_ds_psp(input_size):   
    inputs = Input(shape=input_size)
    conv1 = UnetConv2D(inputs, 32, is_batchnorm=True, name='conv1')
    # conv1 = Dropout(0.2,name='drop_conv1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = UnetConv2D(pool1, 64, is_batchnorm=True, name='conv2')
    # conv2 = Dropout(0.2,name='drop_conv2')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = UnetConv2D(pool2, 128, is_batchnorm=True, name='conv3')
    # conv3 = Dropout(0.1,name='drop_conv3')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = UnetConv2D(pool3, 64, is_batchnorm=True, name='conv4')
    # conv4 = Dropout(0.1, name='drop_conv4')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    center = UnetConv2D(pool4, 512, is_batchnorm=True, name='center')
    
    g1 = UnetGatingSignal(center, is_batchnorm=True, name='g1')
    attn1 = AttnGatingBlock(conv4, g1, 128, '_1')
    up1 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(center), attn1], name='up1')
    
    g2 = UnetGatingSignal(up1, is_batchnorm=True, name='g2')
    attn2 = AttnGatingBlock(conv3, g2, 64, '_2')
    up2 = concatenate([Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up1), attn2], name='up2')

    g3 = UnetGatingSignal(up1, is_batchnorm=True, name='g3')
    attn3 = AttnGatingBlock(conv2, g3, 32, '_3')
    up3 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up2), attn3], name='up3')

    up4 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up3), conv1], name='up4')

    


    psp1  = Conv2DTranspose(256, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2, name='psp1')(center)
    conv6 = UnetConv2D(up1, 256, is_batchnorm=True, name='conv6')
    add1  = add([psp1, conv6])
    psp2  = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2, name='psp2')(add1)
    conv7 = UnetConv2D(up2, 128, is_batchnorm=True, name='conv7')
    add2  = add([psp2, conv7])
    psp3  = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2, name='psp3')(add2)
    conv8 = UnetConv2D(up3, 64, is_batchnorm=True, name='conv8')
    add3  = add([psp3, conv8])
    psp4  = Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2, name='psp4')(add3)
    conv9 = UnetConv2D(up4, 32, is_batchnorm=True, name='conv9')
    add4  = add([psp4, conv9])

    out6 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='pred1')(add1)
    out7 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='pred2')(add2)
    out8 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='pred3')(add3)
    out9 = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='final')(add4)


    model = Model(inputs=[inputs], outputs=[out6, out7, out8, out9])
    loss = {'pred1':focal_tversky,
            'pred2':focal_tversky,
            'pred3':focal_tversky,
            'final':focal_tversky}
    
    loss_weights = {'pred1':1,
                    'pred2':1,
                    'pred3':1,
                    'final':1}
    model.compile(optimizer=SGD(lr=0.1, momentum=0.9, nesterov=True,), loss=loss, metrics=[dice_coef,jaccard_distance,AUC(num_thresholds=500)])
    return model
################################################################################
#####model_11: Attention_Unet_input pyramid_version
def expend_as(tensor, rep,name):
	my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3), arguments={'repnum': rep},  name='psi_up'+name)(tensor)
	return my_repeat


def AttnGatingBlock(x, g, inter_shape, name):
    ''' take g which is the spatially smaller signal, do a conv to get the same
    number of feature channels as x (bigger spatially)
    do a conv on x to also get same geature channels (theta_x)
    then, upsample g to be same size as x 
    add x and g (concat_xg)
    relu, 1x1 conv, then sigmoid then upsample the final - this gives us attn coefficients'''
    
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same', name='xl'+name, kernel_initializer=ki, kernel_regularizer=l2)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv2D(inter_shape, (1, 1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(g)
    upsample_g = Conv2DTranspose(inter_shape, (3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),padding='same', name='g_up'+name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same', name='psi'+name, kernel_initializer=ki, kernel_regularizer=l2)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = expend_as(upsample_psi, shape_x[3],  name)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv2D(shape_x[3], (1, 1), padding='same',name='q_attn_conv'+name, kernel_initializer=ki, kernel_regularizer=l2)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetConv2D(input, outdim, is_batchnorm, name):
	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_1')(input)
	if is_batchnorm:
		x =BatchNormalization(name=name + '_1_bn')(x)
	x = Activation('relu',name=name + '_1_act')(x)

	x = Conv2D(outdim, (3, 3), strides=(1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_2')(x)
	if is_batchnorm:
		x = BatchNormalization(name=name + '_2_bn')(x)
	x = Activation('relu', name=name + '_2_act')(x)
	return x
	

def UnetGatingSignal(input, is_batchnorm, name):
    ''' this is simply 1x1 convolution, bn, activation '''
    shape = K.int_shape(input)
    x = Conv2D(shape[3] * 1, (1, 1), strides=(1, 1), padding="same",  kernel_initializer=ki, kernel_regularizer=l2, name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name = name + '_act')(x)
    return x

def attn_unet_ip(input_size):   
    inputs = Input(shape=input_size)
    scale_img_2 = AveragePooling2D(pool_size=(2, 2), name='input_scale2')(inputs)
    scale_img_3 = AveragePooling2D(pool_size=(2, 2), name='input_scale3')(scale_img_2)
    scale_img_4 = AveragePooling2D(pool_size=(2, 2), name='input_scale4')(scale_img_3)


    conv1 = UnetConv2D(inputs, 32, is_batchnorm=True, name='conv1')
    # conv1 = Dropout(0.2,name='drop_conv1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    input2 = Conv2D(64, (3, 3), padding='same', activation='relu', name='conv_scale2', kernel_initializer=ki, kernel_regularizer=l2)(scale_img_2)
    input2 = concatenate([input2, pool1], axis=3)
    conv2 = UnetConv2D(pool1, 64, is_batchnorm=True, name='conv2')
    # conv2 = Dropout(0.2,name='drop_conv2')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    input3 = Conv2D(128, (3, 3), padding='same', activation='relu', name='conv_scale3', kernel_initializer=ki, kernel_regularizer=l2)(scale_img_3)
    input3 = concatenate([input3, pool2], axis=3)
    conv3 = UnetConv2D(pool2, 128, is_batchnorm=True, name='conv3')
    # conv3 = Dropout(0.1,name='drop_conv3')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    input4 = Conv2D(64, (3, 3), padding='same', activation='relu', name='conv_scale4', kernel_initializer=ki, kernel_regularizer=l2)(scale_img_4)
    input4 = concatenate([input4, pool3], axis=3)
    conv4 = UnetConv2D(pool3, 256, is_batchnorm=True, name='conv4')
    # conv4 = Dropout(0.1, name='drop_conv4')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    center = UnetConv2D(pool4, 512, is_batchnorm=True, name='center')
    
    g1 = UnetGatingSignal(center, is_batchnorm=True, name='g1')
    attn1 = AttnGatingBlock(conv4, g1, 128, '_1')
    up1 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(center), attn1], name='up1')
    
    g2 = UnetGatingSignal(up1, is_batchnorm=True, name='g2')
    attn2 = AttnGatingBlock(conv3, g2, 64, '_2')
    up2 = concatenate([Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up1), attn2], name='up2')

    g3 = UnetGatingSignal(up1, is_batchnorm=True, name='g3')
    attn3 = AttnGatingBlock(conv2, g3, 32, '_3')
    up3 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up2), attn3], name='up3')

    up4 = concatenate([Conv2DTranspose(32, (3,3), strides=(2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(up3), conv1], name='up4')

    out = Conv2D(1, (1, 1), activation='sigmoid', kernel_initializer=ki, name='final')(up4)

    model = Model(inputs=[inputs], outputs=[out])
    loss =[focal_tversky]
    model.compile(optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True), loss=loss, metrics=[dice_coef,dice_coefficient,jaccard_distance,AUC(num_thresholds=500)])
    return model
################################################################################
# from tensorflow.keras.applications.vgg19 import VGG19
def squeeze_excite_block(input, ratio=8):
    init = input
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer=ki, use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer=ki, use_bias=False)(se)

    x = Multiply()([init, se])
    return x

def convolution_block(input, filters):
    x = input
    x = Conv2D(filters, (3, 3), padding="same", kernel_initializer=ki)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same", kernel_initializer=ki)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = squeeze_excite_block(x)
    x = global_context_block(x, reduction_ratio=8, transform_activation='sigmoid')
    return x

def encoder1(input):
    num_filters = [32, 64, 128, 256]
    skip_connections = []
    x = input

    for i, f in enumerate(num_filters):
        x = convolution_block(x, f)
        skip_connections.append(x)
        x = MaxPooling2D((2, 2))(x)
    return x,skip_connections

def decoder1(input, skip_connections):
    num_filters = [256, 128, 64, 32]
    skip_connections.reverse()
    x = input

    for i, f in enumerate(num_filters):
        x = Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same', kernel_initializer=ki)(x)
        x = concatenate([x, skip_connections[i]])
        x = convolution_block(x, f)

    return x

def encoder2(input):
    num_filters = [32, 64, 128, 256]
    skip_connections = []
    x = input

    for i, f in enumerate(num_filters):
        x = convolution_block(x, f)
        skip_connections.append(x)
        x = MaxPooling2D((2, 2))(x)

    return x, skip_connections

def decoder2(input, skip_1, skip_2):
    num_filters = [256, 128, 64, 32]
    skip_2.reverse()
    x = input

    for i, f in enumerate(num_filters):
        x = Conv2DTranspose(f, (2, 2), strides=(2, 2), padding='same', kernel_initializer=ki)(x)
        x = concatenate([x, skip_1[i], skip_2[i]])
        x = convolution_block(x, f)

    return x

def output_block(input,name):
    x = Conv2D(1, (1, 1), padding="same", kernel_initializer=ki)(input)
    x = Activation('sigmoid', name=name)(x)
    return x

def double_unet(shape):
    input = Input(shape)
    pre_out1, skip_1= encoder1(input)
    out_enc1 = convolution_block(pre_out1, 512)
    x = decoder1(out_enc1, skip_1)
    outputs1 = output_block(x,'out_unet1')

    x = input * outputs1
    x = concatenate([input, x],axis=-1)

    pre_out2, skip_2 = encoder2(x)
    out_enc2 = convolution_block(pre_out2, 512)
    x = decoder2(out_enc2, skip_1, skip_2)
    outputs2 = output_block(x,'final')
    model = Model(input, outputs2)
    loss =[focal_tversky]


    model.compile(optimizer=Nadam(lr=0.0001), loss=loss, metrics=[dice_coefficient])
    return model
################################################################################
"""
Standard UNet++ [Zhou et.al, 2018]
Total params: 9,041,601
"""
def standard_unit(input_tensor, stage, nb_filter, kernel_size=3,dropout_rate=0):

    act = 'elu'

    x = Conv2D(nb_filter, (kernel_size, kernel_size), name='conv'+stage+'_1', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(act)(x)
    x = Dropout(dropout_rate, name='dp'+stage+'_1')(x)
    x = Conv2D(nb_filter, (kernel_size, kernel_size), name='conv'+stage+'_2', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(x)
    x = BatchNormalization()(x)
    x = Activation(act)(x)
    x = Dropout(dropout_rate, name='dp'+stage+'_2')(x)
    x = squeeze_excite_block(x)
    return x
def nest_unet(shape, num_class=1, deep_supervision=False):

    nb_filter = [32,64,128,256,512]
    act = 'elu'

    img_input = Input(shape)

    conv1_1 = standard_unit(img_input, stage='11', nb_filter=nb_filter[0])
    pool1 = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(conv1_1)

    conv2_1 = standard_unit(pool1, stage='21', nb_filter=nb_filter[1])
    pool2 = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(conv2_1)

    up1_2 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up12', padding='same')(conv2_1)
    conv1_2 = concatenate([up1_2, conv1_1], name='merge12')
    conv1_2 = standard_unit(conv1_2, stage='12', nb_filter=nb_filter[0])

    conv3_1 = standard_unit(pool2, stage='31', nb_filter=nb_filter[2])
    pool3 = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(conv3_1)

    up2_2 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up22', padding='same')(conv3_1)
    conv2_2 = concatenate([up2_2, conv2_1], name='merge22')
    conv2_2 = standard_unit(conv2_2, stage='22', nb_filter=nb_filter[1])

    up1_3 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up13', padding='same')(conv2_2)
    conv1_3 = concatenate([up1_3, conv1_1, conv1_2], name='merge13')
    conv1_3 = standard_unit(conv1_3, stage='13', nb_filter=nb_filter[0])

    conv4_1 = standard_unit(pool3, stage='41', nb_filter=nb_filter[3])
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(conv4_1)

    up3_2 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up32', padding='same')(conv4_1)
    conv3_2 = concatenate([up3_2, conv3_1], name='merge32')
    conv3_2 = standard_unit(conv3_2, stage='32', nb_filter=nb_filter[2])

    up2_3 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up23', padding='same')(conv3_2)
    conv2_3 = concatenate([up2_3, conv2_1, conv2_2], name='merge23')
    conv2_3 = standard_unit(conv2_3, stage='23', nb_filter=nb_filter[1])

    up1_4 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up14', padding='same')(conv2_3)
    conv1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3], name='merge14')
    conv1_4 = standard_unit(conv1_4, stage='14', nb_filter=nb_filter[0])

    conv5_1 = standard_unit(pool4, stage='51', nb_filter=nb_filter[4])

    up4_2 = Conv2DTranspose(nb_filter[3], (2, 2), strides=(2, 2), name='up42', padding='same')(conv5_1)
    conv4_2 = concatenate([up4_2, conv4_1], name='merge42')
    conv4_2 = standard_unit(conv4_2, stage='42', nb_filter=nb_filter[3])

    up3_3 = Conv2DTranspose(nb_filter[2], (2, 2), strides=(2, 2), name='up33', padding='same')(conv4_2)
    conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33')
    conv3_3 = standard_unit(conv3_3, stage='33', nb_filter=nb_filter[2])

    up2_4 = Conv2DTranspose(nb_filter[1], (2, 2), strides=(2, 2), name='up24', padding='same')(conv3_3)
    conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], name='merge24')
    conv2_4 = standard_unit(conv2_4, stage='24', nb_filter=nb_filter[1])

    up1_5 = Conv2DTranspose(nb_filter[0], (2, 2), strides=(2, 2), name='up15', padding='same')(conv2_4)
    conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], name='merge15')
    conv1_5 = standard_unit(conv1_5, stage='15', nb_filter=nb_filter[0])

    # nestunet_output_1 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_1', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(conv1_2)
    # nestunet_output_2 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_2', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(conv1_3)
    # nestunet_output_3 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_3', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(conv1_4)
    nestunet_output_final= Conv2D(num_class, (1, 1), activation='sigmoid', name='output_final', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(conv1_5)

    if deep_supervision:
        model = Model(img_input, outputs=[nestunet_output_1,
                                         nestunet_output_2,
                                         nestunet_output_3,
                                         nestunet_output_final])
        loss = {'output_1':focal_tversky,
                'output_2':focal_tversky,
                'output_3':focal_tversky,
                'output_final':focal_tversky}
    
        loss_weights = {'output_1':1,
                        'output_2':1,
                        'output_3':1,
                        'output_final':1}
        model.compile(optimizer=Nadam(lr=0.0001), loss=loss, metrics=[dice_coef,dice_coefficient,jaccard_distance,AUC(num_thresholds=500)])
    else:
        model = Model(img_input, outputs=[nestunet_output_final])
        loss =[focal_tversky]
        model.compile(optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True), loss=loss, metrics=[dice_coefficient])
    return model
################################################################################
def inception_unet(input_shape = (240,240,2), 
                        n_labels = 1, 
                        numFilters = 32, 
                        output_mode="sigmoid"):
    
    inputs = Input(input_shape)
    
    conv1 = InceptionModule(inputs, numFilters, kernel_initializer= ki)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = InceptionModule(pool1, 2*numFilters, kernel_initializer= ki)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = InceptionModule(pool2, 4*numFilters, kernel_initializer= ki)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = InceptionModule(pool3, 8*numFilters, kernel_initializer= ki)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = InceptionModule(pool4,16*numFilters, kernel_initializer= ki)

    up6 = Conv2DTranspose(8*numFilters, kernel_size=(3,3),strides=(2, 2), padding='same', kernel_initializer= ki)(conv5)
    up6 = InceptionModule(up6, 8*numFilters, kernel_initializer= ki)
    merge6 = concatenate([conv4,up6],axis=3)
    
    up7 = Conv2DTranspose(4*numFilters, kernel_size=(3,3),strides=(2, 2), padding='same', kernel_initializer= ki)(merge6)
    up7 = InceptionModule(up7, 4*numFilters, kernel_initializer= ki)
    merge7 = concatenate([conv3,up7],axis=3)
    
    up8 = Conv2DTranspose(2*numFilters, kernel_size=(3,3),strides=(2, 2), padding='same', kernel_initializer= ki)(merge7)
    up8 = InceptionModule(up8, 2*numFilters, kernel_initializer= ki)
    merge8 = concatenate([conv2,up8],axis=3)
    
    up9 = Conv2DTranspose(numFilters, kernel_size=(3,3),strides=(2, 2), padding='same', kernel_initializer= ki)(merge8)
    up9 = InceptionModule(up9, numFilters, kernel_initializer= ki)
    merge9 = concatenate([conv1,up9],axis=3)
    
    conv10 = Convolution2D(n_labels, (1,1),  padding = 'same', kernel_initializer = ki)(merge9)
    conv10 = BatchNormalization()(conv10)
    outputs = Activation(output_mode)(conv10)
    
    model = Model(inputs=[inputs], outputs=[outputs], name='inception_unet')
    loss =[focal_tversky]
    model.compile(optimizer=SGD(lr=0.0001, nesterov=True, momentum=0.9), loss=loss, metrics=[dice_coefficient])
    return model
################################################################################
#####model: Unet_3D  
def Unet_3D(input_shape, cout=1, final_activ='sigmoid'):
    def conv_block(input_tensor, num_filters):
        encoder = Conv3D(num_filters, (3, 3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(input_tensor)
        encoder = BatchNormalization()(encoder)
        encoder = Activation('relu')(encoder)
        # encoder = Dropout(0.5)(encoder)
        encoder = Conv3D(num_filters, (3, 3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(encoder)
        encoder = BatchNormalization()(encoder)
        encoder = Activation('relu')(encoder)
        return encoder
    def encoder_block(input_tensor, num_filters):
        encoder = conv_block(input_tensor, num_filters)
        # encoder = Dropout(drop_rate)(encoder)
        encoder_pool = MaxPooling3D((2, 2, 2), strides=(2, 2, 2))(encoder)

        return encoder_pool, encoder

    def decoder_block(input_tensor, concat_tensor, num_filters):
        decoder = Conv3DTranspose(num_filters, (2, 2, 2), strides=(2, 2, 2), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(input_tensor)
        decoder = concatenate([concat_tensor, decoder], axis=-1)
        decoder = BatchNormalization()(decoder)
        decoder = Activation('relu')(decoder)
        # decoder = Dropout(0.2)(decoder)
        decoder = Conv3D(num_filters, (3, 3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(decoder)
        decoder = BatchNormalization()(decoder)
        decoder = Activation('relu')(decoder)
        decoder = Conv3D(num_filters, (3, 3, 3), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(decoder)
        decoder = BatchNormalization()(decoder)
        decoder = Activation('relu')(decoder)
        return decoder 
    inputs = Input(shape=input_shape) 
    encoder0_pool, encoder0 = encoder_block(inputs, 32) 
    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)  
    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) 
    encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) 
    center = conv_block(encoder3_pool, 512) 
    decoder3 = decoder_block(center, encoder3, 256) 
    decoder2 = decoder_block(decoder3, encoder2, 128) 
    decoder1 = decoder_block(decoder2, encoder1, 64) 
    decoder0 = decoder_block(decoder1, encoder0, 32) 
    outputs = Conv3D(cout, (1, 1, 1), activation=final_activ, kernel_initializer=ki, kernel_regularizer=l2)(decoder0) 

    model = Model(inputs=[inputs], outputs=[outputs], name='Unet_3D')
    output= outputs
    model.compile(optimizer=SGD(learning_rate=0.01,momentum=0.9, nesterov=True), loss=[focal_tversky], metrics=[dice_coefficient_correct])
    return model
###############################################################################
"""
Standard UNet++ 3D version[Zhou et.al, 2018]

"""
def squeeze_excite_block(input, ratio=16, activation='relu'):
    init = input
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, 1, filters)

    se = GlobalAveragePooling3D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation=activation, kernel_initializer=ki, use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer=ki, use_bias=False)(se)

    x = Multiply()([init, se])
    return x
def standard_unit(input_tensor, stage, nb_filter, kernel_size=3,dropout_rate=0):

    act = 'elu'

    x = Conv3D(nb_filter, (kernel_size, kernel_size, kernel_size), name='conv'+stage+'_1', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation(act)(x)
    x = Dropout(dropout_rate, name='dp'+stage+'_1')(x)
    x = Conv3D(nb_filter, (kernel_size, kernel_size, kernel_size), name='conv'+stage+'_2', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(x)
    x = BatchNormalization()(x)
    x = Activation(act)(x)
    x = Dropout(dropout_rate, name='dp'+stage+'_2')(x)
    x = squeeze_excite_block(x)
    return x
def Nested_Unet_3D(shape, num_class=1, deep_supervision=False):

    nb_filter = [16,32,64,128,256]
    act = 'elu'

    img_input = Input(shape)

    conv1_1 = standard_unit(img_input, stage='11', nb_filter=nb_filter[0])
    pool1 = MaxPooling3D((2, 2, 2), strides=(2, 2, 2), name='pool1')(conv1_1)

    conv2_1 = standard_unit(pool1, stage='21', nb_filter=nb_filter[1])
    pool2 = MaxPooling3D((2, 2, 2), strides=(2, 2, 2), name='pool2')(conv2_1)

    up1_2 = Conv3DTranspose(nb_filter[0], (2, 2, 2), strides=(2, 2, 2), name='up12', padding='same')(conv2_1)
    conv1_2 = concatenate([up1_2, conv1_1], name='merge12')
    conv1_2 = standard_unit(conv1_2, stage='12', nb_filter=nb_filter[0])

    conv3_1 = standard_unit(pool2, stage='31', nb_filter=nb_filter[2])
    pool3 = MaxPooling3D((2, 2, 2), strides=(2, 2, 2), name='pool3')(conv3_1)

    up2_2 = Conv3DTranspose(nb_filter[1], (2, 2, 2), strides=(2, 2, 2), name='up22', padding='same')(conv3_1)
    conv2_2 = concatenate([up2_2, conv2_1], name='merge22')
    conv2_2 = standard_unit(conv2_2, stage='22', nb_filter=nb_filter[1])

    up1_3 = Conv3DTranspose(nb_filter[0], (2, 2, 2), strides=(2, 2, 2), name='up13', padding='same')(conv2_2)
    conv1_3 = concatenate([up1_3, conv1_1, conv1_2], name='merge13')
    conv1_3 = standard_unit(conv1_3, stage='13', nb_filter=nb_filter[0])

    conv4_1 = standard_unit(pool3, stage='41', nb_filter=nb_filter[3])
    pool4 = MaxPooling3D((2, 2, 2), strides=(2, 2, 2), name='pool4')(conv4_1)

    up3_2 = Conv3DTranspose(nb_filter[2], (2, 2, 2), strides=(2, 2, 2), name='up32', padding='same')(conv4_1)
    conv3_2 = concatenate([up3_2, conv3_1], name='merge32')
    conv3_2 = standard_unit(conv3_2, stage='32', nb_filter=nb_filter[2])

    up2_3 = Conv3DTranspose(nb_filter[1], (2, 2, 2), strides=(2, 2, 2), name='up23', padding='same')(conv3_2)
    conv2_3 = concatenate([up2_3, conv2_1, conv2_2], name='merge23')
    conv2_3 = standard_unit(conv2_3, stage='23', nb_filter=nb_filter[1])

    up1_4 = Conv3DTranspose(nb_filter[0], (2, 2, 2), strides=(2, 2, 2), name='up14', padding='same')(conv2_3)
    conv1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3], name='merge14')
    conv1_4 = standard_unit(conv1_4, stage='14', nb_filter=nb_filter[0])

    conv5_1 = standard_unit(pool4, stage='51', nb_filter=nb_filter[4])

    up4_2 = Conv3DTranspose(nb_filter[3], (2, 2, 2), strides=(2, 2, 2), name='up42', padding='same')(conv5_1)
    conv4_2 = concatenate([up4_2, conv4_1], name='merge42')
    conv4_2 = standard_unit(conv4_2, stage='42', nb_filter=nb_filter[3])

    up3_3 = Conv3DTranspose(nb_filter[2], (2, 2, 2), strides=(2, 2, 2), name='up33', padding='same')(conv4_2)
    conv3_3 = concatenate([up3_3, conv3_1, conv3_2], name='merge33')
    conv3_3 = standard_unit(conv3_3, stage='33', nb_filter=nb_filter[2])

    up2_4 = Conv3DTranspose(nb_filter[1], (2, 2, 2), strides=(2, 2, 2), name='up24', padding='same')(conv3_3)
    conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], name='merge24')
    conv2_4 = standard_unit(conv2_4, stage='24', nb_filter=nb_filter[1])

    up1_5 = Conv3DTranspose(nb_filter[0], (2, 2, 2), strides=(2, 2, 2), name='up15', padding='same')(conv2_4)
    conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], name='merge15')
    conv1_5 = standard_unit(conv1_5, stage='15', nb_filter=nb_filter[0])

    # nestunet_output_1 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_1', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(conv1_2)
    # nestunet_output_2 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_2', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(conv1_3)
    # nestunet_output_3 = Conv2D(num_class, (1, 1), activation='sigmoid', name='output_3', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(conv1_4)
    nestunet_output_final= Conv3D(num_class, (1, 1, 1), activation='sigmoid', name='output_final', kernel_initializer = ki, padding='same', kernel_regularizer=l2)(conv1_5)

    if deep_supervision:
        model = Model(img_input, outputs=[nestunet_output_1,
                                         nestunet_output_2,
                                         nestunet_output_3,
                                         nestunet_output_final])
        loss = {'output_1':focal_tversky,
                'output_2':focal_tversky,
                'output_3':focal_tversky,
                'output_final':focal_tversky}
    
        loss_weights = {'output_1':1,
                        'output_2':1,
                        'output_3':1,
                        'output_final':1}
        model.compile(optimizer=Nadam(lr=0.0001), loss=loss, metrics=[dice_coef,dice_coefficient,jaccard_distance,AUC(num_thresholds=500)])
    else:
        model = Model(img_input, outputs=[nestunet_output_final])
        loss =[focal_tversky]
        model.compile(optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True), loss=loss, metrics=[dice_coefficient_correct])
    return model
###############################################################################
##model: Attention_Unet_version_3D
def squeeze_excite_block(input, ratio=16, activation='relu'):
    init = input
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, 1, filters)

    se = GlobalAveragePooling3D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation=activation, kernel_initializer=ki, use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer=ki, use_bias=False)(se)

    x = Multiply()([init, se])
    return x
def expend_as_3D(tensor, rep,name):
	my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=-1), arguments={'repnum': rep},  name='psi_up'+name)(tensor)
	return my_repeat


def AttnGatingBlock_3D(x, g, inter_shape, name):
    ''' take g which is the spatially smaller signal, do a conv to get the same
    number of feature channels as x (bigger spatially)
    do a conv on x to also get same geature channels (theta_x)
    then, upsample g to be same size as x 
    add x and g (concat_xg)
    relu, 1x1 conv, then sigmoid then upsample the final - this gives us attn coefficients'''
    
    shape_x = K.int_shape(x)  # 32
    shape_g = K.int_shape(g)  # 16

    theta_x = Conv3D(inter_shape, (2, 2, 2), strides=(2, 2, 2), padding='same', name='xl'+name, kernel_initializer=ki, kernel_regularizer=l2)(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

    phi_g = Conv3D(inter_shape, (1, 1, 1), padding='same', kernel_initializer=ki, kernel_regularizer=l2)(g)
    upsample_g = Conv3DTranspose(inter_shape, (3, 3, 3),strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2], shape_theta_x[3] // shape_g[3]),padding='same', name='g_up'+name)(phi_g)  # 16

    concat_xg = add([upsample_g, theta_x])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv3D(1, (1, 1, 1), padding='same', name='psi'+name, kernel_initializer=ki, kernel_regularizer=l2)(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = UpSampling3D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2], shape_x[3] // shape_sigmoid[3]))(sigmoid_xg)  # 32

    upsample_psi = expend_as_3D(upsample_psi, shape_x[-1],  name)
    y = multiply([upsample_psi, x], name='q_attn'+name)

    result = Conv3D(shape_x[-1], (1, 1, 1), padding='same',name='q_attn_conv'+name, kernel_initializer=ki, kernel_regularizer=l2)(y)
    result_bn = BatchNormalization(name='q_attn_bn'+name)(result)
    return result_bn

def UnetConv3D(input, outdim, is_batchnorm, name):
    x = Conv3D(outdim, (3, 3, 3), strides=(1, 1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_1')(input)
    if is_batchnorm:
       x =BatchNormalization(name=name + '_1_bn')(x)
    x = Activation('relu',name=name + '_1_act')(x)
    #x = DropBlock3D(keep_prob=0.9,block_size=3)(x)
    x = Conv3D(outdim, (3, 3, 3), strides=(1, 1, 1), kernel_initializer=ki, kernel_regularizer=l2, padding="same", name=name+'_2')(x)
    if is_batchnorm:
       x = BatchNormalization(name=name + '_2_bn')(x)
    x = Activation('relu', name=name + '_2_act')(x)
    return x
	

def UnetGatingSignal_3D(input, is_batchnorm, name):
    ''' this is simply 1x1 convolution, bn, activation '''
    shape = K.int_shape(input)
    x = Conv3D(shape[-1] * 1, (1, 1, 1), strides=(1, 1, 1), padding="same",  kernel_initializer=ki, kernel_regularizer=l2, name=name + '_conv')(input)
    if is_batchnorm:
        x = BatchNormalization(name=name + '_bn')(x)
    x = Activation('relu', name = name + '_act')(x)
    return x

# plain old attention gates in u-net, NO multi-input, NO deep supervision
def attn_unet_3D(input_size):   
    inputs = Input(shape=input_size)

    conv1 = UnetConv3D(inputs, 32, is_batchnorm=True, name='conv1')
    # conv1 = Dropout(0.2,name='drop_conv1')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = UnetConv3D(pool1, 64, is_batchnorm=True, name='conv2')
    # conv2 = Dropout(0.2,name='drop_conv2')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = UnetConv3D(pool2, 128, is_batchnorm=True, name='conv3')
    # conv3 = Dropout(0.1,name='drop_conv3')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)

    conv4 = UnetConv3D(pool3, 256, is_batchnorm=True, name='conv4')
    # conv4 = Dropout(0.1, name='drop_conv4')(conv4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(conv4)
    
    center = UnetConv3D(pool4, 512, is_batchnorm=True, name='center')

    g1 = UnetGatingSignal_3D(center, is_batchnorm=True, name='g1')
    attn1 = AttnGatingBlock_3D(conv4, g1, 256, '_1')
    # attn1_se = squeeze_excite_block(attn1)
    up1 = concatenate([Conv3DTranspose(256, (3,3,3), strides=(2,2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(center), attn1], name='up1')
    conv1_d = UnetConv3D(up1, 256, is_batchnorm=True, name='conv1_d')

    g2 = UnetGatingSignal_3D(conv1_d, is_batchnorm=True, name='g2')
    attn2 = AttnGatingBlock_3D(conv3, g2, 128, '_2')
    # attn2_se = squeeze_excite_block(attn2)
    up2 = concatenate([Conv3DTranspose(128, (3,3,3), strides=(2,2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(conv1_d), attn2], name='up2')
    conv2_d= UnetConv3D(up2, 128, is_batchnorm=True, name='conv2_d')

    g3 = UnetGatingSignal_3D(conv2_d, is_batchnorm=True, name='g3')
    attn3 = AttnGatingBlock_3D(conv2, g3, 64, '_3')
    # attn3_se = squeeze_excite_block(attn3)
    up3 = concatenate([Conv3DTranspose(64, (3,3,3), strides=(2,2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(conv2_d), attn3], name='up3')
    conv3_d= UnetConv3D(up3, 64, is_batchnorm=True, name='conv3_d')

    up4 = concatenate([Conv3DTranspose(32, (3,3,3), strides=(2,2,2), padding='same', activation='relu', kernel_initializer=ki, kernel_regularizer=l2)(conv3_d), conv1], name='up4')
    conv4_d= UnetConv3D(up4, 32, is_batchnorm=True, name='conv4_d')

    out = Conv3D(1, (1, 1, 1), activation='sigmoid',  kernel_initializer=ki, name='final')(conv4_d)
    
    model = Model(inputs=[inputs], outputs=[out])
    model.compile(optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True), loss=[focal_tversky], metrics=[dice_coefficient_correct])
    return model
################################################################################
def step_decay(epoch):
   initial_lrate = 0.0001
   drop = 0.8
   epochs_drop = 5.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate
lrate = LearningRateScheduler(step_decay)


csv = CSVLogger("/gdrive/My Drive/nima_log/new_log_my_ms_data_3D/chrome_mahmoodian acount/Attention_Unet_3D/1/1.log", separator=',', append=True)
filepath = "/gdrive/My Drive/nima_log/new_log_my_ms_data_3D/chrome_mahmoodian acount/Attention_Unet_3D/1/saved-model-3D-MS-{epoch:02d}-{loss:.4f}-{dice_coefficient_correct:.4f}.hdf5"
checkpoint =ModelCheckpoint(filepath,monitor='dice_coefficient_correct', verbose=1, save_weights_only=True,save_best_only=False, mode='max',period=5)
callbacks=[checkpoint, csv]


model = attn_unet_3D(input_shape)
print(model.summary())


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 64,  0                                            
__________________________________________________________________________________________________
conv1_1 (Conv3D)                (None, 64, 64, 64, 3 2624        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_1_bn (BatchNormalization) (None, 64, 64, 64, 3 128         conv1_1[0][0]                    
__________________________________________________________________________________________________
conv1_1_act (Activation)        (None, 64, 64, 64, 3 0           conv1_1_bn[0][0]                 
______________________________________________________________________________________________

In [ ]:
my_model= model.fit(trainset_generator_3D_ISBI_aug_random_patch_equal_prob(7,1),steps_per_epoch=27,epochs=500,shuffle=False,verbose=1,initial_epoch=475,callbacks=callbacks)

Epoch 476/500
27/27 [==============================] - 1045s 39s/step - loss: 0.5117 - dice_coefficient_correct: 0.8983
Epoch 477/500
27/27 [==============================] - 229s 9s/step - loss: 0.5170 - dice_coefficient_correct: 0.8946
Epoch 478/500
27/27 [==============================] - 231s 9s/step - loss: 0.5238 - dice_coefficient_correct: 0.8909
Epoch 479/500
27/27 [==============================] - 231s 9s/step - loss: 0.5298 - dice_coefficient_correct: 0.8791
Epoch 480/500
27/27 [==============================] - 230s 9s/step - loss: 0.5152 - dice_coefficient_correct: 0.8950

Epoch 00480: saving model to /gdrive/My Drive/nima_log/new_log_my_ms_data_3D/chrome_mahmoodian acount/Attention_Unet_3D/1/saved-model-3D-MS-480-0.5203-0.8883.hdf5
Epoch 481/500
27/27 [==============================] - 232s 9s/step - loss: 0.5276 - dice_coefficient_correct: 0.8861
Epoch 482/500
27/27 [==============================] - 233s 9s/step - loss: 0.5260 - dice_coefficient_correct: 0.8809
Epoch 48

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, precision_recall_curve
model.load_weights('/gdrive/My Drive/nima_log/new_log_my_ms_data_3D/chrome_mahmoodian acount/Attention_Unet_3D/1/saved-model-3D-MS-475-0.5348-0.8795.hdf5')